# Example for downloading Land Cover Data from GLOBE API

In [1]:
# Import Libraries
import pandas as pd
import requests
import json
import datetime
import csv
from pathlib import Path

In [4]:
def getLCData():
    # Define the GLOBE API request URL
    base_url = "https://api.globe.gov/search/v1/measurement/protocol/measureddate/"
    requestParameters = {
        "protocols": "land_covers",
        "startdate": "2018-09-01",
        "enddate" : datetime.datetime.utcnow().date(),
        "geojson" : "FALSE",
        "sample" : "FALSE"
    }
    # Make the request
    response = requests.get(base_url, params=requestParameters)
    # Keep the results
    results = response.json()["results"]

    # Pass the results as a Dataframe
    df = pd.DataFrame(results)
    # Expand the nested 'data' column by listing the contents and passing as a new dataframe
    df = pd.concat([df, pd.DataFrame(list(df['data']))], axis=1)
    #Drop the previously nested data column
    df.drop(columns=['data'], inplace=True)
    # Define the protocol short name used for column labels
    shortname = requestParameters['protocols'].replace("_", "")
    #Rename/Shorten Columns for complete display
    df.columns = df.columns.str.replace(shortname, '', regex=False)
    df = df[df["DataSource"]=='GLOBE Observer App']
    # Replace NA values with 'null' string
    df = df.fillna('null')
    return df

In [5]:
#LC
lc = getLCData()
#Write to CSV
outputFile = Path("../input_data/Land_Cover_Data_AllYears_AllFields.csv")
lc.to_csv(outputFile, sep=',', index=False, encoding='utf-8', quoting=csv.QUOTE_ALL, quotechar='"', escapechar='"')